In [1]:
import dgl
import copy
import torch
import random
import numpy as np
import pandas as pd
from model import RGCN_NET
from sklearn.metrics import accuracy_score
from sklearn import metrics


def setup_seed(seed):
    """
    fix the random seed
    :param seed: the random seed
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    return None

setup_seed(4)




# attribute
name2id = {}
id2name = {}


x = pd.read_csv("data/features.csv")

for i in range(len(x)):
    name = x.loc[i][0]
    id2name[i] = name
    name2id[name] = i

x = x.iloc[:, 1:]
x = x.to_numpy()




adj = pd.read_csv("data/Network.csv")

max_id = x.shape[0]
edges_1 = []
edges_2 = []
for i in range(len(adj)):
    start = adj.loc[i][0]
    mid = adj.loc[i][1]
    end = adj.loc[i][2]
    # if start != "NA":
    #     try:
    #         name2id[start]
    #     except:
    #         name2id[start] = max_id
    #         id2name[max_id] = start
    #         max_id += 1
    #
    # if mid != "NA":
    #     try:
    #         name2id[mid]
    #     except:
    #         name2id[mid] = max_id
    #         id2name[max_id] = mid
    #         max_id += 1
    #
    # if end != "NA":
    #     try:
    #         name2id[end]
    #     except:
    #         name2id[end] = max_id
    #         id2name[max_id] = end
    #         max_id += 1

    if 1:
        try:
            # 1
            edges_1.append([name2id[start], name2id[mid]])
        except:
            pass
        # try:
        #     edges_1.append([name2id[mid], name2id[start]])
        # except:
        #     pass
    if 1:
        try:
            edges_2.append([name2id[mid], name2id[end]])
        except:
            pass
        # try:
        #     edges_2.append([name2id[end], name2id[mid]])
        # except:
        #     pass



label = pd.read_csv("data/label.csv")
# final 0.6936936936936937



y = np.zeros((x.shape[0], ))
for i in range(len(label["node"])):
    try:
        index = name2id[label["node"].iloc[i]]
        y[index] = label["label"].iloc[i]
    except:
        pass

y = torch.tensor(y).float()
#print(y)

# x = np.concatenate([x, np.zeros((len(name2id)-x.shape[0], 5))], axis=0)
x = torch.tensor(x).float()

print(edges_1)
g = dgl.heterograph(
    {
        ('rna', 'type1', 'rna'): edges_1,
        ('rna', 'type2', 'rna'): edges_2
    }
)

print(g)

# logits = rgcn_net(None, None)
# y_pred = (logits >= 0.5).int().squeeze(dim=-1)

# acc = accuracy_score(y[test_id], y_pred[test_id])
# print(acc)

# print(emb["inc_rna"].shape)
# print(emb["m_rna"].shape)

#
# GCN

idx = list(range(x.shape[0]))
random.shuffle(idx)
idx = np.array(idx)


[[88, 213], [101, 166], [88, 122], [306, 134], [306, 134], [306, 130], [306, 130], [337, 432], [303, 107], [303, 217], [257, 144], [368, 114], [368, 433], [352, 126], [364, 198], [364, 198], [329, 162], [369, 170], [84, 154], [103, 118], [103, 118], [370, 176], [332, 176], [370, 176], [332, 176], [371, 147], [371, 147], [364, 432], [364, 432], [257, 112], [372, 111], [73, 106], [265, 434], [373, 435], [373, 435], [373, 435], [373, 435], [374, 143], [374, 143], [16, 204], [20, 171], [375, 178], [84, 149], [88, 127], [89, 105], [89, 105], [101, 166], [101, 127], [376, 136], [377, 160], [250, 145], [257, 126], [259, 176], [265, 149], [265, 149], [268, 122], [277, 151], [277, 436], [285, 200], [304, 149], [307, 184], [307, 184], [307, 184], [307, 184], [378, 126], [323, 205], [325, 437], [325, 140], [326, 126], [326, 126], [326, 126], [352, 126], [355, 435], [355, 208], [364, 133], [379, 128], [46, 126], [88, 446], [380, 128], [381, 112], [374, 122], [382, 206], [383, 128], [257, 144], [25

C:\Users\ASUS\AppData\Local\Temp\ipykernel_29776\3235661770.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  name = x.loc[i][0]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_29776\3235661770.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  start = adj.loc[i][0]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_29776\3235661770.py:57: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mid = adj.loc[i][1]
C:\Users\ASUS\AppData\Local\Temp\ipy

In [12]:

from sklearn.model_selection import KFold
import numpy as np
kf = KFold(n_splits=5,shuffle=False)  # 初始化KFold
for train_id, test_id in kf.split(x):
    #print('train_index:%s , test_index: %s ' %(train_id, test_id))

    criterion = torch.nn.BCELoss()
    rgcn_net = RGCN_NET(g, 400, 1, dropout=0)
    optimizer = torch.optim.Adam(rgcn_net.parameters(), lr=1 * 1e-2)

    best_acc = 0
    best_model = None
    threshold = 0.3
    best_logit = 0
    for epoch in range(100):
        rgcn_net.eval()
        logits = rgcn_net(None, None)

        y_pred = (logits >= threshold).int().squeeze(dim=-1)
        acc = accuracy_score(y[test_id], y_pred[test_id])
        #print(acc)
        if best_acc < acc:
            best_acc = acc
            best_model = copy.deepcopy(rgcn_net)

        rgcn_net.train()
        logits = rgcn_net(None, None)
        loss = criterion(logits.squeeze(dim=-1)[train_id], y[train_id])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    print(best_acc)
    best_model.eval()
    logits = best_model(None, None)

    y_pred = (logits >= threshold).int().squeeze(dim=-1)
    acc = accuracy_score(y[test_id], y_pred[test_id])





    logits = best_model(None, None)
    y_pred = (logits >= threshold).int().squeeze(dim=-1)
    logits_list = logits[test_id][:, 0].data
    index = sorted(range(len(logits_list)), key=lambda k: logits_list[k], reverse=True)
    # print(y_pred[index])


    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y[test_id], y_pred[test_id])


    acc = accuracy_score(y[test_id], y_pred[test_id])
    print("acc", acc)

    import seaborn as sns
    import pandas as pd
    import matplotlib.pyplot as plt
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, classification_report

    y_true = y[test_id]
    y_pred = y_pred[test_id]
    # 1.计算混淆矩阵
    cm = confusion_matrix(y_true, y_pred)
    # cm = np.array([[40, 24],
    #  [ 9, 38]])

    conf_matrix = pd.DataFrame(cm, index=['0', '1'], columns=['0', '1'])  # 数据有5个类别


    #
    # # 2.计算accuracy
    # print('accuracy_score', accuracy_score(y_true, y_pred))
    #
    # # 3.计算多分类的precision、recall、f1-score分数
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)
    auc = metrics.auc(fpr, tpr)

    print('precision', precision_score(y_true, y_pred))
    print('recall', recall_score(y_true, y_pred))
    print('f1-score', f1_score(y_true, y_pred))
    print('auc', auc)
    print('#####')

    #
    # # 下面这个可以显示出每个类别的precision、recall、f1-score。
    # print('classification_report\n', classification_report(y_true, y_pred))


0.8181818181818182
acc 0.8181818181818182
precision 0.5833333333333334
recall 0.6363636363636364
f1-score 0.6086956521739131
auc 0.7532467532467532
#####
0.7777777777777778
acc 0.7777777777777778
precision 0.7755102040816326
recall 1.0
f1-score 0.8735632183908045
auc 0.5217391304347826
#####
0.797979797979798
acc 0.797979797979798
precision 0.7419354838709677
recall 0.92
f1-score 0.8214285714285714
auc 0.7967346938775511
#####
0.5612244897959183
acc 0.5612244897959183
precision 0.32727272727272727
recall 0.75
f1-score 0.4556962025316456
auc 0.625
#####
0.5714285714285714
acc 0.5714285714285714
precision 0.43243243243243246
recall 1.0
f1-score 0.6037735849056604
auc 0.6818181818181819
#####
